This is an example of a simple CNN developed, trained and utilized

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [1]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import numpy as np

In [2]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.
train_dir = "D:/CODE/25-26/CS 3B/CHRISTIAN_PAULO_PILLADO/Exercise_6/dataset/train"  # e.g. './muffin-vs-chihuahua/train'
test_dir = "D:/CODE/25-26/CS 3B/CHRISTIAN_PAULO_PILLADO/Exercise_6/dataset/test"    # e.g. './muffin-vs-chihuahua/test'

In [3]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [4]:
# DATA PREPROCESSING & AUGMENTATION
from tensorflow.keras.applications.resnet50 import preprocess_input

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # ResNet-specific preprocessing
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 575 images belonging to 2 classes.
Found 143 images belonging to 2 classes.
Found 106 images belonging to 2 classes.


In [5]:
# RESNET50 MODEL ARCHITECTURE with Transfer Learning
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import regularizers

# Load pretrained ResNet50 without top layers
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
)

# Freeze the base model layers (transfer learning)
base_model.trainable = False

# Build model on top of ResNet50
model_resnet = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

# Compile with lower learning rate for transfer learning
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model_resnet.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model_resnet.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,145,281 (92.11 MB)

 Trainable params: 557,569 (2.13 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
# TRAINING THE RESNET MODEL
history_resnet = model_resnet.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 105s 5s/step - accuracy: 0.7235 - loss: 1.1973 - val_accuracy: 0.9231 - val_loss: 0.8708
Epoch 2/10
 1/18 ━━━━━━━━━━━━━━━━━━━━ 1:16 4s/step - accuracy: 0.8125 - loss: 1.1759

In [ ]:
# EVALUATE THE RESNET MODEL
test_loss_resnet, test_acc_resnet = model_resnet.evaluate(test_generator)
print(f"\n=== RESNET50 MODEL RESULTS ===")
print(f"Test Accuracy: {test_acc_resnet:.4f}")
print(f"Test Loss: {test_loss_resnet:.4f}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.9906 - loss: 0.6388

=== RESNET50 MODEL RESULTS ===
Test Accuracy: 0.9906
Test Loss: 0.6388


In [ ]:
# SAVE THE RESNET MODEL
model_resnet.save('exercise_6_resnet_pillado.h5')
print("ResNet model saved as 'exercise_6_resnet_pillado.h5'")

ResNet model saved as 'exercise_6_resnet_PILLADO.h5'


In [ ]:
# RESNET INFERENCE SCRIPT
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input

def predict_image_resnet(img_path, model):
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img)
    img_array = preprocess_input(img_array)  # ResNet preprocessing
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array, verbose=0)[0,0]
    label = "pizza" if pred >= 0.5 else "icecream"
    confidence = pred if pred >= 0.5 else (1 - pred)
    return label, confidence, pred

# RUN 1: Predictions with ResNet model
print("=== RUN 1: RESNET MODEL PREDICTIONS ===")
test_images = [
    "D:/CODE/25-26/CS 3B/CHRISTIAN_PAULO_PILLADO/Exercise_6/predict/img5.jpg",
    "D:/CODE/25-26/CS 3B/CHRISTIAN_PAULO_PILLADO/Exercise_6/predict/img6.jpg",
    "D:/CODE/25-26/CS 3B/CHRISTIAN_PAULO_PILLADO/Exercise_6/predict/img7.jpg",
    "D:/CODE/25-26/CS 3B/CHRISTIAN_PAULO_PILLADO/Exercise_6/predict/img8.jpg"
]

for i, img_path in enumerate(test_images, 1):
    label, confidence, raw_pred = predict_image_resnet(img_path, model_resnet)
    print(f"Image {i}: {label} (Confidence: {confidence:.2%}, Raw: {raw_pred:.4f})")

# RUN 2: Second run
print("\n=== RUN 2: RESNET MODEL PREDICTIONS (Second Run) ===")
for i, img_path in enumerate(test_images, 1):
    label, confidence, raw_pred = predict_image_resnet(img_path, model_resnet)
    print(f"Image {i}: {label} (Confidence: {confidence:.2%}, Raw: {raw_pred:.4f})")

=== RUN 1: RESNET MODEL PREDICTIONS ===
Image 1: pizza (Confidence: 91.69%, Raw: 0.9169)
Image 2: icecream (Confidence: 99.99%, Raw: 0.0001)
Image 3: pizza (Confidence: 99.95%, Raw: 0.9995)
Image 4: icecream (Confidence: 99.75%, Raw: 0.0025)

=== RUN 2: RESNET MODEL PREDICTIONS (Second Run) ===
Image 1: pizza (Confidence: 91.69%, Raw: 0.9169)
Image 2: icecream (Confidence: 99.99%, Raw: 0.0001)
Image 3: pizza (Confidence: 99.95%, Raw: 0.9995)
Image 4: icecream (Confidence: 99.75%, Raw: 0.0025)
